In [ ]:
!pip install tensorflow faiss-cpu numpy opencv-python


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
import faiss


In [ ]:

# 画像からembeddingを抽出するモデルの設定
model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

def get_image_embedding(img_path):
    """
    指定されたパスの画像からembeddingを取得します。
    """
    # 画像の読み込みと前処理
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # 画像からembeddingを取得
    embedding = model.predict(img_array)
    return embedding

def load_images_and_embeddings(directory):
    """
    指定されたディレクトリ内の全ての画像を読み込み、それぞれのembeddingを取得します。
    """
    image_paths = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith(('.jpg', '.jpeg', '.png'))]
    embeddings = []
    for img_path in image_paths:
        embedding = get_image_embedding(img_path)
        embeddings.append(embedding)
    return image_paths, np.vstack(embeddings)

def create_faiss_index(embeddings):
    """
    embeddingをFAISSインデックスに追加します。
    """
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index


def search_similar_images(query_img_path, image_paths, index, top_k=5):
    """
    クエリ画像に対して、FAISSインデックスを使用して類似画像を検索します。
    """
    query_embedding = get_image_embedding(query_img_path)
    distances, indices = index.search(query_embedding, top_k)
    return [(image_paths[i], distances[0][j]) for j, i in enumerate(indices[0])]



In [ ]:

# メイン処理
if __name__ == "__main__":
    # 画像が保存されているディレクトリ
    image_directory = '/content/image-for-embedding/'

    # 画像とembeddingをロード
    image_paths, embeddings = load_images_and_embeddings(image_directory)

    # FAISSインデックスを作成
    index = create_faiss_index(embeddings)



In [ ]:
    # クエリ画像のパス
    query_image_path = '/content/target-image/target.jpeg'

    # # 類似画像を検索
    # similar_images = search_similar_images(query_image_path, image_paths, index)
    similar_images = search_similar_images(query_image_path, image_paths, index, top_k=5)

    # 結果を表示
    print("Query image:", query_image_path)
    print("Similar images:")
    for img_path in similar_images:
        print(img_path)

    # # 結果を表示
    # print("Query image:", query_image_path)
    # print("Top 5 similar images with their distances:")
    # for img_path, distance in similar_images:
    #     print(f"Image: {img_path}, Distance: {distance}")

